In [1]:
############### Import packages
import os, numpy as np, pandas as pd, time, glob, re, math, statsmodels.api as sm, patsy as ps
from tqdm import tqdm
from time import process_time
from datetime import datetime
from datetime import date
from openpyxl import load_workbook
from patsy import dmatrices

############### Set working directory to parent directory
if os.getcwd() != 'F:\\github\\narrative_conservatism\\code':
    os.chdir('F:\\github\\narrative_conservatism\\code')

############### Set pandas column printing constraint
pd.set_option('display.max_columns', None)

In [2]:
########################################################################################
############ Merge IBES quarterly data with CRSP_COMP_EDGAR data #######################
########################################################################################

########### Read IBES raw data file
ibes_cols = ['cusip8', 'cname', 'fpi', 'value', 'fpedats', 'anndats', 'actual', 'anndats_act']
ibes = pd.read_csv('..\\filings\\ibes.csv', usecols = ibes_cols)
print('number of cusip-fpedats-analyst: ' + str(ibes.shape[0]))

### Reorder and rename IBES columns
ibes = ibes[['cusip8', 'cname', 'fpedats', 'value', 'actual', 'anndats_act', 'anndats', 'fpi']]
ibes = ibes.rename(columns={'cusip8': 'cusip'})

### correct fpedats format
ibes['fpedats'] = ibes['fpedats'].str.replace('jan','01')
ibes['fpedats'] = ibes['fpedats'].str.replace('feb','02')
ibes['fpedats'] = ibes['fpedats'].str.replace('mar','03')
ibes['fpedats'] = ibes['fpedats'].str.replace('apr','04')
ibes['fpedats'] = ibes['fpedats'].str.replace('may','05')
ibes['fpedats'] = ibes['fpedats'].str.replace('jun','06')
ibes['fpedats'] = ibes['fpedats'].str.replace('jul','07')
ibes['fpedats'] = ibes['fpedats'].str.replace('aug','08')
ibes['fpedats'] = ibes['fpedats'].str.replace('sep','09')
ibes['fpedats'] = ibes['fpedats'].str.replace('oct','10')
ibes['fpedats'] = ibes['fpedats'].str.replace('nov','11')
ibes['fpedats'] = ibes['fpedats'].str.replace('dec','12')
ibes['fpedats'] = pd.to_datetime(ibes['fpedats'], format='%d%m%Y')

print(ibes['fpedats'].describe())

### Delete missing cusip8, actual
del_cusip = ibes[ibes['cusip'].isnull()].shape[0]
ibes = ibes[ibes['cusip'].isnull() == False]
print('number of obs. that contain missing cusip: ' + str(del_cusip))

del_actual = ibes[ibes['actual'].isnull()].shape[0]
ibes = ibes[ibes['actual'].isnull() == False]
print('number of obs. that contain missing actual: ' + str(del_actual))

### aggregate by cusip-fpedats and get actual, median, afe and consensus from ibes raw data
ibes_css = ibes.groupby(['cusip', 'fpedats'])['actual'].median().to_frame()
ibes_css['median'] = ibes.groupby(['cusip', 'fpedats'])['value'].median().to_frame()
ibes_css['afe'] = ibes_css['actual'] - ibes_css['median']
ibes_css['consensus'] = ibes.groupby(['cusip', 'fpedats'])['value'].mean().to_frame()
ibes_css['leap_consensus'] = ibes_css.groupby(['cusip'])['consensus'].shift(-1)

### make a consensus forecast dataset by cusip-fpedats
ibes = ibes.loc[ibes.duplicated(subset=['cusip', 'fpedats']) == False, ['cusip','fpedats']]
ibes = ibes.join(ibes_css, on=['cusip', 'fpedats'])

### create merge date_key - %y only
ibes['date_key'] = ibes['fpedats'].astype(str).str[:-6].astype(int)

print('number of cusip-fpedats, dropping missing cusip and actual: ' + str(ibes.shape[0]))

number of cusip-fpedats-analyst: 9812071
count                 9812071
unique                    446
top       2015-12-31 00:00:00
freq                   426980
first     1981-12-31 00:00:00
last      2019-07-31 00:00:00
Name: fpedats, dtype: object
number of obs. that contain missing cusip: 60781
number of obs. that contain missing actual: 1305946
number of cusip-fpedats, dropping missing cusip and actual: 155539


In [13]:
##################### Read EDGAR_CRSP_COMP and creat merge date_key
crsp_comp_edgar = pd.read_csv('..\\filings\\id_crsp_comp_text_10-Q.csv')
crsp_comp_edgar['date_key'] = crsp_comp_edgar['fyearq'].astype(int)

##################### inner merge EDGAR_CRSP_COMP and IBES, key not unique in both datasets
crsp_comp_edgar_ibes = pd.merge(crsp_comp_edgar, ibes, on = ['cusip', 'date_key'], how='inner', validate = 'm:m')
print('number of cusip-date after merging: ' + str(crsp_comp_edgar_ibes.shape[0]))

number of cusip-date after merging: 110111


In [14]:
########################################################################################
############ Merge CRSP_COMP_EDGAR_IBES data with SEG data #############################
########################################################################################

########### Read SEG raw data file
seg_cols = ['gvkey', 'stype', 'sid', 'datadate', 'snms', 'cusip', 'cik']
seg = pd.read_csv('..\\filings\\compustat_seg.csv', usecols = seg_cols)
print('number of cusip-fpedats-analyst: ' + str(seg.shape[0]))

### Reorder and rename IBES columns
seg = seg[['gvkey', 'cik', 'cusip', 'datadate', 'stype', 'sid', 'snms']]
seg = seg.rename(columns={'datadate': 'date_comp'})

### parse date_comp format
seg['date_comp'] = pd.to_datetime(seg['date_comp'], format='%Y%m%d')

### aggregate by gvkey-date_comp-stype and get count of sid
seg_count = seg.groupby(['gvkey', 'date_comp', 'stype'])['sid'].count().to_frame()

### make a segment dataset by gvkey-date_comp-stype
seg = seg.loc[seg.duplicated(subset=['gvkey', 'date_comp', 'stype']) == False, ['gvkey', 'date_comp', 'stype']]
seg = seg.join(seg_count, on=['gvkey', 'date_comp', 'stype'])
seg_bus = seg[seg['stype']=='BUSSEG']
seg_bus = seg_bus.drop(columns=['stype'])
seg_bus = seg_bus.rename(columns={'sid': 'nseg_bus'})
seg_geo = seg[seg['stype']=='GEOSEG']
seg_geo = seg_geo.drop(columns=['stype'])
seg_geo = seg_geo.rename(columns={'sid': 'nseg_geo'})
seg = seg.loc[seg.duplicated(subset=['gvkey', 'date_comp']) == False, ['gvkey', 'date_comp']]
seg = pd.merge(seg, seg_bus, on = ['gvkey', 'date_comp'], how='left', validate = '1:1')
seg = pd.merge(seg, seg_geo, on = ['gvkey', 'date_comp'], how='left', validate = '1:1')
seg = seg.sort_values(by=['gvkey', 'date_comp'])
seg.loc[seg['nseg_geo'].isnull(), 'nseg_geo'] = 1
seg.loc[seg['nseg_bus'].isnull(), 'nseg_bus'] = 1
seg['nseg_bus'] = seg['nseg_bus'].astype(int)
seg['nseg_geo'] = seg['nseg_geo'].astype(int)

### create merge date_key - %y only
seg['date_key'] = seg['date_comp'].astype(str).str[:-6].astype(int)
seg = seg.drop(columns=['date_comp'])

print('number of gvkey-datadate in SEGMENT data: ' + str(seg.shape[0]))

number of cusip-fpedats-analyst: 452653
number of gvkey-datadate in SEGMENT data: 50876


In [15]:
##################### left merge EDGAR_CRSP_COMP_IBES and SEG, key not unique in both datasets
crsp_comp_edgar_ibes_seg_left = pd.merge(crsp_comp_edgar_ibes, seg, on = ['gvkey', 'date_key'], how='left', validate = 'm:m')
print('Number of gvkey-quarter obs. after merging, left: '+ str(crsp_comp_edgar_ibes_seg_left.shape[0]))
# crsp_comp_edgar_ibes_seg_inner = pd.merge(crsp_comp_edgar_ibes, seg, on = ['gvkey', 'date_key'], how='inner', validate = 'm:m')
# print('Number of gvkey-quarter obs. after merging, inner: '+ str(crsp_comp_edgar_ibes_seg_inner.shape[0]))

##################### fill missing segments in crsp_comp_edgar_ibes_seg_left with 1
crsp_comp_edgar_ibes_seg_left.loc[crsp_comp_edgar_ibes_seg_left['nseg_bus'].isnull(), 'nseg_bus'] = 1
crsp_comp_edgar_ibes_seg_left.loc[crsp_comp_edgar_ibes_seg_left['nseg_geo'].isnull(), 'nseg_geo'] = 1

Number of gvkey-quarter obs. after merging, left: 110130


In [16]:
########################################################################################
############################### Variable Creation ######################################
########################################################################################

################### CRSP_COMP_EDGAR_IBES_SEG_LEFT ######################################

######## AGE: log(1 + age from the first year the firm entered the CRSP dataset)
crsp_comp_edgar_ibes_seg_left['AGE'] = np.log(1 + crsp_comp_edgar_ibes_seg_left['age'])
######## BUSSEG: log(1 + number of business segments), or 1 if item is missing from Compustat; and
######## GEOSEG: log(1 + number of geographic segments), or 1 if item is missing from Compustat.
crsp_comp_edgar_ibes_seg_left['BUSSEG'] = np.log(1 + crsp_comp_edgar_ibes_seg_left['nseg_bus'])
crsp_comp_edgar_ibes_seg_left['GEOSEG'] = np.log(1 + crsp_comp_edgar_ibes_seg_left['nseg_geo'])
######## AFE, analyst forecast error, \
######## defined as I/B/E/S earnings per share minus the median of the most recent analysts’ forecasts, \
######## deflated by stock price per share at the end of the fiscal quarter
crsp_comp_edgar_ibes_seg_left['AFE'] = crsp_comp_edgar_ibes_seg_left['afe']/crsp_comp_edgar_ibes_seg_left['prccq']
######## AF, analyst consensus forecast for one-year-ahead earnings per share, scaled by stock price per share at the end of the fiscal quarter
crsp_comp_edgar_ibes_seg_left['AF'] = crsp_comp_edgar_ibes_seg_left['leap_consensus']/crsp_comp_edgar_ibes_seg_left['prccq']

# ################### CRSP_COMP_EDGAR_IBES_SEG_INNER ######################################

# ######## AGE: log(1 + age from the first year the firm entered the CRSP dataset)
# crsp_comp_edgar_ibes_seg_inner['AGE'] = np.log(1 + crsp_comp_edgar_ibes_seg_inner['age'])
# ######## BUSSEG: log(1 + number of business segments), or 1 if item is missing from Compustat; and
# ######## GEOSEG: log(1 + number of geographic segments), or 1 if item is missing from Compustat.
# crsp_comp_edgar_ibes_seg_inner['BUSSEG'] = np.log(1 + crsp_comp_edgar_ibes_seg_inner['nseg_bus'])
# crsp_comp_edgar_ibes_seg_inner['GEOSEG'] = np.log(1 + crsp_comp_edgar_ibes_seg_inner['nseg_geo'])
# ######## AFE, analyst forecast error, \
# ######## defined as I/B/E/S earnings per share minus the median of the most recent analysts’ forecasts, \
# ######## deflated by stock price per share at the end of the fiscal quarter
# crsp_comp_edgar_ibes_seg_inner['AFE'] = crsp_comp_edgar_ibes_seg_inner['afe']/crsp_comp_edgar_ibes_seg_inner['prccq']
# ######## AF, analyst consensus forecast for one-year-ahead earnings per share, scaled by stock price per share at the end of the fiscal quarter
# crsp_comp_edgar_ibes_seg_inner['AF'] = crsp_comp_edgar_ibes_seg_inner['leap_consensus']/crsp_comp_edgar_ibes_seg_inner['prccq']

In [17]:
########################################################################################
########################## Variable Screening (LEFT) ###################################
########################################################################################

########## Drop files (firm-quarter) that have positive/negative infinity DEARN
del_DEARN = crsp_comp_edgar_ibes_seg_left[(crsp_comp_edgar_ibes_seg_left['DEARN']==np.inf) \
                                          | (crsp_comp_edgar_ibes_seg_left['DEARN']==-np.inf)].shape[0]
crsp_comp_edgar_ibes_seg_left = crsp_comp_edgar_ibes_seg_left[(crsp_comp_edgar_ibes_seg_left['DEARN']!=np.inf) \
                                                              & (crsp_comp_edgar_ibes_seg_left['DEARN']!=-np.inf)]
print('number of files that have positive/negative infinity DEARN: ' + str(del_DEARN))

### Drop missing AF
del_AF = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['AF'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg_left = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['AF'].isnull() == False]
print('number of obs. that contain missing AF: ' + str(del_AF))

### Drop missing AFE
del_AFE = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['AFE'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg_left = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['AFE'].isnull() == False]
print('number of obs. that contain missing AFE: ' + str(del_AFE))

### Drop missing EARN
del_EARN = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['EARN'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg_left = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['EARN'].isnull() == False]
print('number of obs. that contain missing EARN: ' + str(del_EARN))

### Drop missing STD_EARN
del_STD_EARN = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['STD_EARN'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg_left = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['STD_EARN'].isnull() == False]
print('number of obs. that contain missing STD_EARN: ' + str(del_STD_EARN))

### Drop missing EARN
del_DEARN = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['DEARN'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg_left = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['DEARN'].isnull() == False]
print('number of obs. that contain missing DEARN: ' + str(del_DEARN))

############## Inspect sample size after variable screening
print('Number of firm-quarters after variable screening: ' + str(crsp_comp_edgar_ibes_seg_left.shape[0]))

############## Winsorize ##############################
###### Define a function that winsorize a variable at 1% and 99% 
def winsorize (df, colnames):
    for col in colnames:
        varq01 = df[col].quantile(.01)
        varq99 = df[col].quantile(.99)
        df[col] = df[col].clip(varq01, varq99)
    return df

############## Winsorize TABLE 1 (line 1) and TABLE 4 (line 2) variables
crsp_comp_edgar_ibes_seg_left = winsorize(crsp_comp_edgar_ibes_seg_left, \
                                          ['AF', 'AFE', 'BUSSEG','GEOSEG','AGE','EARN', 'DEARN', 'STD_EARN', 'STD_RET', \
                                          'CFO', 'leap1_EARN', 'leap2_EARN', 'leap3_EARN', 'leap1_CFO', 'leap2_CFO', 'leap3_CFO'])

number of files that have positive/negative infinity DEARN: 0
number of obs. that contain missing AF: 15241
number of obs. that contain missing AFE: 0
number of obs. that contain missing EARN: 64
number of obs. that contain missing STD_EARN: 3162
number of obs. that contain missing DEARN: 0
Number of firm-quarters after variable screening: 91663


In [35]:
######## Create ABTONE: residual from TONE regression 
y, X = ps.dmatrices('TONE ~ 1 + EARN + RET + SIZE + MTB + STD_RET + STD_EARN + AGE + BUSSEG + GEOSEG + LOSS + DEARN + AFE + AF', \
                    data = crsp_comp_edgar_ibes_seg_left, return_type = 'dataframe')
model = sm.OLS(y, X)
res = model.fit()
# res.summary()
crsp_comp_edgar_ibes_seg_left['ABTONE'] = res.resid

######## Create abtone for REPLICATION: residual from tone regression 
y, X = ps.dmatrices('tone ~ 1 + EARN + RET + SIZE + MTB + STD_RET + STD_EARN + AGE + BUSSEG + GEOSEG + LOSS + DEARN + AFE + AF', \
                    data = crsp_comp_edgar_ibes_seg_left, return_type = 'dataframe')
model = sm.OLS(y, X)
res = model.fit()
# res.summary()
crsp_comp_edgar_ibes_seg_left['abtone'] = res.resid

####### Drop duplicated gykey-cquarter
crsp_comp_edgar_ibes_seg_left = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left.duplicated(subset=['gvkey', 'cquarter']) == False]

############## Save merged ID_CRSP_COMP_TEXT to csv file
crsp_comp_edgar_ibes_seg_left.to_csv('..\\filings\\crsp_comp_edgar_ibes_seg_10-Q.csv', index = 0)

In [20]:
# ########################################################################################
# ########################## Variable Screening (INNER) ##################################
# ########################################################################################

# ########## Drop files (firm-quarter) that have positive/negative infinity DEARN
# del_DEARN = crsp_comp_edgar_ibes_seg_inner[(crsp_comp_edgar_ibes_seg_inner['DEARN']==np.inf) | (crsp_comp_edgar_ibes_seg_inner['DEARN']==-np.inf)].shape[0]
# crsp_comp_edgar_ibes_seg_inner = crsp_comp_edgar_ibes_seg_inner[(crsp_comp_edgar_ibes_seg_inner['DEARN']!=np.inf) & (crsp_comp_edgar_ibes_seg_inner['DEARN']!=-np.inf)]
# print('number of files that have positive/negative infinity DEARN: ' + str(del_DEARN))

# ### Drop missing AF
# del_AF = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['AF'].isnull()].shape[0]
# crsp_comp_edgar_ibes_seg_inner = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['AF'].isnull() == False]
# print('number of obs. that contain missing AF: ' + str(del_AF))

# ### Drop missing AFE
# del_AFE = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['AFE'].isnull()].shape[0]
# crsp_comp_edgar_ibes_seg_inner = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['AFE'].isnull() == False]
# print('number of obs. that contain missing AFE: ' + str(del_AFE))

# ### Drop missing EARN
# del_EARN = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['EARN'].isnull()].shape[0]
# crsp_comp_edgar_ibes_seg_inner = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['EARN'].isnull() == False]
# print('number of obs. that contain missing EARN: ' + str(del_EARN))

# ### Drop missing STD_EARN
# del_STD_EARN = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['STD_EARN'].isnull()].shape[0]
# crsp_comp_edgar_ibes_seg_inner = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['STD_EARN'].isnull() == False]
# print('number of obs. that contain missing STD_EARN: ' + str(del_STD_EARN))

# ### Drop missing EARN
# del_DEARN = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['DEARN'].isnull()].shape[0]
# crsp_comp_edgar_ibes_seg_inner = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['DEARN'].isnull() == False]
# print('number of obs. that contain missing DEARN: ' + str(del_DEARN))

# ############## Inspect sample size after variable screening
# print('Number of firm-quarters after variable screening: ' + str(crsp_comp_edgar_ibes_seg_inner.shape[0]))

# ############## Winsorize SIZE, MTB, LEV
# ###### Define a function that winsorize a variable at 1% and 99% 
# def winsorize (df, colnames):
#     for col in colnames:
#         varq01 = df[col].quantile(.01)
#         varq99 = df[col].quantile(.99)
#         df[col] = df[col].clip(varq01, varq99)
#     return df

# crsp_comp_edgar_ibes_seg_inner = winsorize(crsp_comp_edgar_ibes_seg_inner, ['AF', 'AFE', 'BUSSEG','GEOSEG','AGE','EARN', 'DEARN', 'STD_EARN', 'STD_RET'])

In [21]:
# ######## Create ABTONE for INNER: residual from TONE regression
# y, X = ps.dmatrices('TONE ~ 1 + EARN + RET + SIZE + MTB + STD_RET + STD_EARN + AGE + BUSSEG + GEOSEG + LOSS + DEARN + AFE + AF', \
#                     data = crsp_comp_edgar_ibes_seg_inner, return_type = 'dataframe')
# model = sm.OLS(y,X)
# res = model.fit()
# # res.summary()
# crsp_comp_edgar_ibes_seg_inner['ABTONE'] = res.resid

# ############## Save merged ID_CRSP_COMP_TEXT to csv file
# crsp_comp_edgar_ibes_seg_inner.to_csv('..\\filings\\crsp_comp_edgar_ibes_seg_10-Q.csv', index = 0)

In [38]:
########################################################################################
############### Table 3: Summary Statistics and Correlation Matrix #####################
########################################################################################

############# Table 3 Panel A: Summary statistics for selected variables
######### Variable groups:
# 1st line: textual variables, generally consistent with LM's summary statistics
# 2nd line: fundamental variables (main)
# 3rd line: abtone
selected_vars = crsp_comp_edgar_ibes_seg_left[['NW','nw', 'TONE','TLAG', \
                                               'RET', 'NEG', 'SIZE', 'MTB', 'LEV', \
                                               'AF', 'AFE', 'BUSSEG','GEOSEG','AGE','EARN', 'DEARN', 'STD_EARN', 'STD_RET', 'LOSS', 'ABTONE' \
                                             ]]

T3PA = selected_vars.describe().transpose() 

############# Summary statistics for all raw and processed variables
full_summary = crsp_comp_edgar_ibes_seg_left.describe().transpose()

############# Save T3PA
table_path = '..\\output\\Tables.xlsx'
if os.path.exists(table_path) == True:
    book = load_workbook(table_path)
    writer = pd.ExcelWriter(table_path, engine = 'openpyxl')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    T3PA.to_excel(writer, sheet_name='T3PA_raw', float_format="%.4f")

    writer.save()
    writer.close()
    
else:
    T3PA.to_excel(table_path, sheet_name='T3PA_raw', float_format="%.4f")

T3PA

,count,mean,std,min,25%,50%,75%,max
NW,91627.0,8.945670,0.763557,7.044033,8.423542,9.009570,9.477080,13.490002
nw,91627.0,10214.619665,9672.180308,1145.000000,4552.000000,8180.000000,13056.000000,722159.000000
TONE,91627.0,-8.456879,6.884674,-64.542890,-12.434659,-7.472271,-3.641661,22.287390
TLAG,91627.0,39.021893,6.248740,0.000000,36.000000,40.000000,44.000000,52.000000
RET,91627.0,0.018249,0.253121,-1.578704,-0.112627,0.007300,0.129860,4.849226
NEG,91627.0,0.482925,0.499711,0.000000,0.000000,0.000000,1.000000,1.000000
SIZE,91627.0,6.446897,1.776314,2.001958,5.175051,6.317072,7.562965,11.205298
MTB,91627.0,3.515685,4.007446,0.288280,1.485544,2.343638,3.902785,30.875352
LEV,91627.0,0.192382,0.182138,0.000000,0.010619,0.162314,0.315080,0.724226
AF,91627.0,0.043048,0.066427,-0.262005,0.022599,0.048514,0.073176,0.227481


In [39]:
# full_summary

In [40]:
############# Table 3 Panel B: Correlation matrix for selected variables
######### pearson correlation
T3PB_pearson = selected_vars.corr(method='pearson')

# T3PB_pearson

In [41]:
######### spearman correlation
T3PB_spearman = selected_vars.corr(method='spearman')

# T3PB_spearman

In [42]:
######### Combine two correlation matrices. right-up matrix: pearson; left-down matrix: spearman 
for row in list(range(0, len(T3PB_spearman.index))):
    T3PB_spearman.iloc[row, row+1:] = T3PB_pearson.iloc[row, row+1:]
    
##### Save T3PB
table_path = '..\\output\\Tables.xlsx'
if os.path.exists(table_path) == True:
    book = load_workbook(table_path)
    writer = pd.ExcelWriter(table_path, engine = 'openpyxl')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    T3PB_spearman.to_excel(writer, sheet_name='T3PB_raw', float_format="%.4f")

    writer.save()
    writer.close()
    
else:
    T3PB_spearman.to_excel(table_path, sheet_name='T3PB_raw', float_format="%.4f")

T3PB_spearman

,NW,nw,TONE,TLAG,RET,NEG,SIZE,MTB,LEV,AF,AFE,BUSSEG,GEOSEG,AGE,EARN,DEARN,STD_EARN,STD_RET,LOSS,ABTONE
NW,1.000000,0.792007,-0.461374,-0.195012,-0.007635,0.003238,0.257638,0.058590,0.036694,-0.067150,0.012158,0.238026,0.258270,-0.037606,-0.115315,0.001409,0.089902,-0.033634,0.127678,-0.384247
nw,1.000000,1.000000,-0.349849,-0.092631,-0.012140,0.009689,0.166531,0.047870,0.048756,-0.053547,0.005067,0.145253,0.161717,-0.056438,-0.085966,-0.001502,0.066561,-0.014412,0.092169,-0.298085
TONE,-0.485856,-0.485856,1.000000,0.025286,0.020639,-0.021117,-0.069736,-0.016044,0.069095,0.069470,0.098039,-0.033608,-0.069600,0.053839,0.155804,-0.001923,-0.144036,-0.081175,-0.214718,0.955292
TLAG,-0.266151,-0.266151,0.029061,1.000000,-0.022362,0.034282,-0.330875,-0.021665,0.009275,-0.092252,-0.127323,-0.229433,-0.245725,-0.228526,-0.136737,-0.004589,0.121168,0.189319,0.146165,0.019587
RET,-0.008026,-0.008026,0.028918,-0.032508,1.000000,-0.684116,-0.064348,-0.025886,0.002053,-0.018139,0.154691,-0.012363,-0.020881,0.001867,0.063599,0.036173,0.010535,0.266026,-0.067866,-0.000009
NEG,0.004365,0.004365,-0.024158,0.032958,-0.865520,1.000000,0.000058,0.012506,-0.001911,0.015130,-0.124160,0.000845,0.011854,-0.018047,-0.071425,-0.018781,0.015887,-0.117558,0.077913,-0.000091
SIZE,0.266427,0.266427,-0.052558,-0.332699,-0.024050,-0.001055,1.000000,0.233500,0.099765,0.077028,0.269617,0.197609,0.215094,0.344072,0.258788,-0.023517,-0.197646,-0.309972,-0.263421,-0.000055
MTB,0.048230,0.048230,0.037343,-0.041914,-0.054747,0.032936,0.381733,1.000000,0.045625,-0.156196,0.120164,0.005065,0.002844,-0.088436,-0.041192,0.022194,0.158544,0.035904,0.034348,-0.000075
LEV,0.014771,0.014771,0.074589,-0.000548,0.003059,-0.004083,0.142990,-0.111242,1.000000,0.166920,-0.068269,0.034836,-0.005186,0.101119,0.039378,0.033873,-0.124489,-0.072099,-0.058274,0.068218
AF,-0.017108,-0.017108,0.060526,-0.124703,-0.086487,0.071470,0.025440,-0.299181,0.250857,1.000000,0.057038,0.068250,0.079258,0.202072,0.472181,0.016082,-0.256464,-0.145191,-0.410087,0.000085
